Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [45]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Neural network with L2 loss

In [5]:
batch_size = 128
hidden_units = 1024
beta = .0001
gamma = .001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units]))
    biases1 = tf.Variable(tf.zeros([hidden_units]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_units, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden1 = tf.nn.relu(logits1)
    logits2 = tf.matmul(hidden1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)) + beta*tf.nn.l2_loss(weights1) + gamma*tf.nn.l2_loss(weights2)
  
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)                           
                                     

Let's run it:

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 460.177399
Minibatch accuracy: 8.6%
Validation accuracy: 31.6%
Minibatch loss at step 500: 42.595242
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 35.715340
Minibatch accuracy: 76.6%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 34.041611
Minibatch accuracy: 78.1%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 27.917704
Minibatch accuracy: 85.9%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 31.187037
Minibatch accuracy: 78.1%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 23.071777
Minibatch accuracy: 85.2%
Validation accuracy: 82.7%
Test accuracy: 88.8%


Now for logistic regression model:

In [ ]:
batch_size = 128
beta = .001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
batch_size = 12
beta = .001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
hidden_units = 1024
beta = .01
gamma = .01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units]))
    biases1 = tf.Variable(tf.zeros([hidden_units]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_units, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden1 = tf.nn.dropout(tf.tanh(logits1),.5)
    logits2 = tf.matmul(hidden1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)) + beta*tf.nn.l2_loss(weights1) + gamma*tf.nn.l2_loss(weights2)
  
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.tanh(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.tanh(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)                           
                                     

Let's run it:

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [8]:
batch_size = 128
hidden_units1 = 1024
hidden_units2 = 300
hidden_units3 = 50

alpha = .001
beta = .001
gamma = .01
delta = .01

graph = tf.Graph()
with graph.as_default():

    #global_step = tf.Variable(0)
    #learning_rate = tf.train.exponential_decay(.5, global_step, 3000, 0.96, staircase=True)
    
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units1]))
    biases1 = tf.Variable(tf.zeros([hidden_units1]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_units1, hidden_units2]))
    biases2 = tf.Variable(tf.zeros([hidden_units2]))
    
    weights3 = tf.Variable(tf.truncated_normal([hidden_units2, hidden_units3]))
    biases3 = tf.Variable(tf.zeros([hidden_units3]))
    
    weights4 = tf.Variable(tf.truncated_normal([hidden_units3, num_labels]))
    biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden1 = tf.nn.dropout(tf.tanh(logits1),.5)
    logits2 = tf.matmul(hidden1, weights2) + biases2
    hidden2 = tf.nn.dropout(tf.tanh(logits2),.5)
    logits3 = tf.matmul(hidden2, weights3) + biases3
    hidden3 = tf.nn.dropout(tf.tanh(logits3),.5)
    logits4 = tf.matmul(hidden3, weights4) + biases4

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits4)) + alpha*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2) + gamma*tf.nn.l2_loss(weights3) + delta*tf.nn.l2_loss(weights4)
  
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # f.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits4)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)                   
    

Let's run it:

In [9]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 502.558746
Minibatch accuracy: 10.9%
Validation accuracy: 9.8%
Minibatch loss at step 500: 264.656281
Minibatch accuracy: 46.1%
Validation accuracy: 52.8%
Minibatch loss at step 1000: 160.770218
Minibatch accuracy: 58.6%
Validation accuracy: 63.1%
Minibatch loss at step 1500: 98.061165
Minibatch accuracy: 65.6%
Validation accuracy: 66.6%
Minibatch loss at step 2000: 59.939697
Minibatch accuracy: 72.7%
Validation accuracy: 69.7%
Minibatch loss at step 2500: 36.749519
Minibatch accuracy: 69.5%
Validation accuracy: 73.9%
Minibatch loss at step 3000: 22.566532
Minibatch accuracy: 78.1%
Validation accuracy: 77.8%
Minibatch loss at step 3500: 14.022223
Minibatch accuracy: 80.5%
Validation accuracy: 74.4%
Minibatch loss at step 4000: 8.959866
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 4500: 5.520603
Minibatch accuracy: 88.3%
Validation accuracy: 79.6%
Minibatch loss at step 5000: 3.632371
Minibatch accuracy: 84.4%
Validati

KeyboardInterrupt: 

Second attempt:

In [18]:
batch_size = 128
hidden_units1 = 1024
hidden_units2 = 256
hidden_units3 = 128

#alpha = .001
#beta = .001
#gamma = .01
#delta = .01

graph = tf.Graph()
with graph.as_default():

    global_step = tf.Variable(0)
    
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([hidden_units1]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_units1, hidden_units2], stddev=np.sqrt(2.0 / hidden_units1 )))
    biases2 = tf.Variable(tf.zeros([hidden_units2]))
    
    weights3 = tf.Variable(tf.truncated_normal([hidden_units2, hidden_units3], stddev=np.sqrt(2.0 / hidden_units2 )))
    biases3 = tf.Variable(tf.zeros([hidden_units3]))
    
    weights4 = tf.Variable(tf.truncated_normal([hidden_units3, num_labels], stddev=np.sqrt(2.0 / hidden_units3 )))
    biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden1 = tf.nn.dropout(tf.nn.relu(logits1),.5)
    logits2 = tf.matmul(hidden1, weights2) + biases2
    hidden2 = tf.nn.dropout(tf.nn.relu(logits2),.5)
    logits3 = tf.matmul(hidden2, weights3) + biases3
    hidden3 = tf.nn.dropout(tf.nn.relu(logits3),.5)
    logits4 = tf.matmul(hidden3, weights4) + biases4

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits4)) + alpha*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2) + gamma*tf.nn.l2_loss(weights3) + delta*tf.nn.l2_loss(weights4)
  

  # Optimizer.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) # f.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits4)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)                   
    

Let's run it:

In [19]:
num_steps = 20000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.790567
Minibatch accuracy: 14.8%
Validation accuracy: 26.9%
Minibatch loss at step 500: 1.499308
Minibatch accuracy: 75.8%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 1.104977
Minibatch accuracy: 83.6%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 0.977039
Minibatch accuracy: 82.0%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.857380
Minibatch accuracy: 83.6%
Validation accuracy: 86.1%
Minibatch loss at step 2500: 0.817384
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 0.704889
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%
Minibatch loss at step 3500: 0.673281
Minibatch accuracy: 85.2%
Validation accuracy: 86.7%
Minibatch loss at step 4000: 0.928288
Minibatch accuracy: 82.8%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 0.567880
Minibatch accuracy: 89.8%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 0.565259
Minibatch accuracy: 88.3%
Validation accurac

Third attempt (MNIST database):

In [3]:
import os
os.chdir('/Users/JAustin/Desktop/MNIST')
train = np.genfromtxt('train.csv', delimiter=",", dtype=int)[1:,:]
test = np.genfromtxt('test.csv', delimiter=",", dtype=int)[1:,:]

In [7]:
train_labels = train[:,0]
train_data = np.delete(train,0,1)
pixel_depth = 255.0
train_eq = (train_data - pixel_depth / 2) / pixel_depth
test_eq = (test - pixel_depth / 2) / pixel_depth
train_dataset = train_eq.astype(np.float32)
test_dataset = test_eq.astype(np.float32)

(42000,)


In [15]:
pixel_depth = 255.0
train_eq = (train_data - pixel_depth / 2) / pixel_depth
test_eq = (test - pixel_depth / 2) / pixel_depth
train_dataset = train_eq.astype(np.float32)
test_dataset = test_eq.astype(np.float32)

In [20]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
# valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
#test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
# print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape)

Training set (42000, 784) (42000, 10)
Test set (28000, 784)


In [21]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [22]:
batch_size = 128
hidden_units1 = 1024
hidden_units2 = 256
hidden_units3 = 128

#alpha = .001
#beta = .001
#gamma = .01
#delta = .01

graph = tf.Graph()
with graph.as_default():

    global_step = tf.Variable(0)
    
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    #tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([hidden_units1]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_units1, hidden_units2], stddev=np.sqrt(2.0 / hidden_units1 )))
    biases2 = tf.Variable(tf.zeros([hidden_units2]))
    
    weights3 = tf.Variable(tf.truncated_normal([hidden_units2, hidden_units3], stddev=np.sqrt(2.0 / hidden_units2 )))
    biases3 = tf.Variable(tf.zeros([hidden_units3]))
    
    weights4 = tf.Variable(tf.truncated_normal([hidden_units3, num_labels], stddev=np.sqrt(2.0 / hidden_units3 )))
    biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden1 = tf.nn.dropout(tf.nn.relu(logits1),.5)
    logits2 = tf.matmul(hidden1, weights2) + biases2
    hidden2 = tf.nn.dropout(tf.nn.relu(logits2),.5)
    logits3 = tf.matmul(hidden2, weights3) + biases3
    hidden3 = tf.nn.dropout(tf.nn.relu(logits3),.5)
    logits4 = tf.matmul(hidden3, weights4) + biases4

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits4)) # + alpha*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2) + gamma*tf.nn.l2_loss(weights3) + delta*tf.nn.l2_loss(weights4)
  
    # Saver
    
    saver = tf.train.Saver([weights1, biases1, weights2, biases2, weights3, biases3, weights4, biases4])

  # Optimizer.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) # f.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step) tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits4)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)                   
    

In [23]:
num_steps = 20000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    
    solution_arr = np.zeros((len(test_dataset)+1,2))
    solution_arr[1:,0] = np.linspace(1,len(test_dataset),len(test_dataset))
    solution_arr[1:,1] = np.argmax(test_prediction.eval(), axis=1)
    np.savetxt("results_net.csv", solution_arr, fmt='%i', delimiter=",")
    saver.save(session, 'my-test-model')
    #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.818129
Minibatch accuracy: 10.9%
Minibatch loss at step 2000: 0.276945
Minibatch accuracy: 93.8%
Minibatch loss at step 4000: 0.159384
Minibatch accuracy: 93.8%
Minibatch loss at step 6000: 0.134217
Minibatch accuracy: 94.5%
Minibatch loss at step 8000: 0.184281
Minibatch accuracy: 95.3%
Minibatch loss at step 10000: 0.093245
Minibatch accuracy: 97.7%
Minibatch loss at step 12000: 0.123868
Minibatch accuracy: 98.4%
Minibatch loss at step 14000: 0.202023
Minibatch accuracy: 96.1%
Minibatch loss at step 16000: 0.040518
Minibatch accuracy: 99.2%
Minibatch loss at step 18000: 0.024755
Minibatch accuracy: 99.2%


In [31]:
with tf.Session() as session:
    new_saver = tf.train.import_meta_graph('my-test-model.meta')
    new_saver.restore(session, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    weights1 = graph.get_tensor_by_name("weights1")
    weights1 = graph.get_tensor_by_name("weights1")
    weights1 = graph.get_tensor_by_name("weights1")
    weights1 = graph.get_tensor_by_name("weights1")
    biases1 = graph.get_tensor_by_name("biases1")
    biases2 = graph.get_tensor_by_name("biases2")
    biases3 = graph.get_tensor_by_name("biases3")
    biases4 = graph.get_tensor_by_name("biases4")
    
    tf_test_dataset = tf.constant(test_dataset)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
    solution_arr = np.zeros((len(test_dataset)+1,2))
    solution_arr[1:,0] = np.linspace(1,len(test_dataset),len(test_dataset))
    solution_arr[1:,1] = np.argmax(test_prediction.eval(), axis=1)

INFO:tensorflow:Restoring parameters from ./my-test-model


KeyError: "The name 'weights1:0' refers to a Tensor which does not exist. The operation, 'weights1', does not exist in the graph."

In [1]:
# import tensorflow as tf
import numpy as np
import os

os.chdir('/Users/JAustin/Desktop/MNIST')
train_database = np.genfromtxt('train.csv', delimiter=",", dtype=int)[1:,:]

In [16]:
train_labels = train_database[:,0].reshape(42000,1)
train_data = np.delete(train_database,0,1)
pixel_depth = 255.0
train_eq = (train_data - pixel_depth / 2) / pixel_depth
#test_eq = (test - pixel_depth / 2) / pixel_depth
training_data = train_eq.astype(np.float32)
training_labels = np.eye(10)[np.random.choice(10, np.shape(training_data)[0])]
print(np.shape(training_data), np.shape(training_labels))

# training_data = np.random.rand(20000, 784)
# training_labels = np.eye(10)[np.random.choice(10, np.shape(training_data)[0])]

data_size = np.shape(training_data)[0] # 20000
num_params = np.shape(training_data)[1] # 784
num_classes = np.shape(training_labels)[1] # 10

weights = np.random.rand(num_classes, num_params) # or randn
biases = np.random.rand(num_classes, 1)

(42000, 784) (42000, 10)


In [24]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=1)[:,None]

def predict(data, batch_size, weights, biases):
    index = np.random.choice(len(data), batch_size, replace=False)
    output = np.dot(data[index], weights.T) + biases.T
    return index, softmax(output)

def evaluate(data, labels, weights, biases, batch_size):
    index = np.random.choice(len(data), batch_size, replace=False)
    accuracy = (np.argmax(softmax(np.dot(data[index], weights.T) + biases.T), axis=1)==np.argmax(training_labels[index], axis=1)).sum()*100 / batch_size
    print("The accuracy of your model is %s%%!" % accuracy)
    return accuracy, index

def gradient_descent(data, labels, weights, biases, batch_size, iterations, learning_rate):
    for i in range(iterations):
        index, prediction = predict(data, batch_size, weights, biases)
        loss = - np.tensordot(labels[index], np.log(prediction), axes=2) / batch_size # + .1*np.linalg.norm(weights)**2 # cross entropy loss
        if i % 50 == 0: print("Loss at step %s is %s" % (i, loss))
        dW = np.sum((prediction - labels[index])[..., None] * data[index][:, None, :], axis=0) / batch_size
        dB=np.sum(prediction - labels[index], axis=0)[:,None] / batch_size
        weights = weights - learning_rate[i]*dW
        biases = biases - learning_rate[i]*dB

    return weights, biases

In [39]:
batch_size = 1024
iterations = 4000

# learning_rate = np.array([.5*(1 - x/iterations) for x in range(iterations)]) # linear learning rate
# learning_rate = 1.5*np.exp(-5*np.arange(0,iterations)/iterations) # exponential learning rate
learning_rate = 1.5*np.ones(iterations)

new_weights, new_biases = gradient_descent(training_data, training_labels, weights, biases, batch_size, iterations, learning_rate)
accuracy, index = evaluate(training_data, training_labels, new_weights, new_biases, batch_size) # probably should be larger than batch_size

Loss at step 0 is 7.4306961607
Loss at step 50 is 76.0356967794
Loss at step 100 is 80.142233443
Loss at step 150 is 58.1752172172
Loss at step 200 is 54.1570531266
Loss at step 250 is 42.097933483
Loss at step 300 is 44.1980473386
Loss at step 350 is 43.1708016469
Loss at step 400 is 41.4464206824
Loss at step 450 is 22.8554600903
Loss at step 500 is 37.0392938917
Loss at step 550 is 34.463062084
Loss at step 600 is 35.4272682191
Loss at step 650 is 30.9295749456
Loss at step 700 is 24.765902904
Loss at step 750 is 22.6228226312
Loss at step 800 is 23.1406610155
Loss at step 850 is 18.9429739601
Loss at step 900 is 14.5147050707
Loss at step 950 is 12.9732790779
Loss at step 1000 is 13.3516906776
Loss at step 1050 is 9.28862887033
Loss at step 1100 is 11.1384007157
Loss at step 1150 is 7.86088511092
Loss at step 1200 is 7.66819951561
Loss at step 1250 is 7.14562738438
Loss at step 1300 is 4.58027641854
Loss at step 1350 is 4.04890064004
Loss at step 1400 is 3.94781708349
Loss at step 

In [13]:
np.argmax(softmax(np.dot(training_data[0:4],weights.T) + biases.T), axis=1)

array([2, 8, 5, 2])

In [ ]:
np.argmax(training_labels[0:4], axis=1))